<a href="https://colab.research.google.com/github/seanjudelyons/RMAI_CandidatePack/blob/main/Solution_1_CommentClassification_Candidate_Lyons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!git clone https://github.com/seanjudelyons/RMAI_CandidatePack
%cd RMAI_CandidatePack/Comment Classification

Cloning into 'RMAI_CandidatePack'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
/content/RMAI_CandidatePack/Comment Classification/RMAI_CandidatePack/Comment Classification


In [22]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import numpy as np
import csv
import io
import os
fpath = os.getcwd(); fpath

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'/content/RMAI_CandidatePack/Comment Classification/RMAI_CandidatePack/Comment Classification'

In [23]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

with open(fpath + '/Delay Sample Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      label=float(row[1])
      data = row[0]
      training_sentences.append(data)
      training_labels.append(label)

with open(fpath + '/Delay Scoring Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
      label=float(row[1])
      data = row[0]
      testing_sentences.append(data)
      testing_labels.append(label)

In [24]:
training_sentences = np.array(training_sentences)
training_labels = np.array(training_labels)

testing_sentences = np.array(testing_sentences)
testing_labels = np.array(testing_labels)

In [25]:
print(training_sentences[1000])
print(training_labels[1000])
print(testing_sentences[200])
print(testing_labels[200])

stripping site compound almost complete
0.0
broken machinery
1.0


In [26]:
vocab_size = 2432
embedding_dim = 100
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
print(padded[0])

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   46 1020   95  826
  663 1947]


In [27]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(training_sentences, key=word_count)
length_longest_sentence = len(word_tokenize(longest_sentence))
print("The longest sentence has {} words".format(length_longest_sentence))

The longest sentence has 94 words


In [28]:
print(padded.shape)
print(training_labels.shape)
print(testing_sentences.shape)
print(testing_labels.shape)

(2432, 100)
(2432,)
(608,)
(608,)


In [32]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(padded[3])
print(testing_labels[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? existing concrete footing found whilst excavating sw 140 near pit 11 140
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  141   70  149   26  134   61 1949 1344  216    7
  484 1344]
0.0


In [33]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          243200    
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 60006     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 303,213
Trainable params: 303,213
Non-trainable params: 0
_________________________________________________________________


In [34]:
num_epochs = 1000
model.fit(padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Epoch 1/1000
76/76 [==============================] - 0s 6ms/step - loss: 0.5047 - accuracy: 0.7944 - val_loss: 0.4829 - val_accuracy: 0.7895
Epoch 2/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.3807 - accuracy: 0.8035 - val_loss: 0.3627 - val_accuracy: 0.8372
Epoch 3/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.2479 - accuracy: 0.9062 - val_loss: 0.3604 - val_accuracy: 0.8388
Epoch 4/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.1891 - accuracy: 0.9519 - val_loss: 0.3462 - val_accuracy: 0.8684
Epoch 5/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.1596 - accuracy: 0.9733 - val_loss: 0.3541 - val_accuracy: 0.8701
Epoch 6/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.1401 - accuracy: 0.9873 - val_loss: 0.3736 - val_accuracy: 0.8651
Epoch 7/1000
76/76 [==============================] - 0s 5ms/step - loss: 0.1277 - accuracy: 0.9918 - val_loss: 0.4070 - val_accuracy: 0.8602
Epoch 

In [35]:
s = "Asbestos pipe removed on site"
seq = tokenizer.texts_to_sequences([s])
pad_s = pad_sequences(seq, maxlen=max_length, truncating=trunc_type)
print(pad_s)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0 204  14 131   1   2]]


In [41]:
print(model.predict(pad_s))

[[0.18074602]]


In [42]:
e = model.layers[0]
s=[]
weights = e.get_weights()[0]
for i in training_sentences:
  s.append(i)
sentencelist=np.array(s)

In [43]:
print(weights.shape)
print(sentencelist.shape)

(2432, 100)
(2432,)


In [44]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for count, word in enumerate(sentencelist):
  embeddings = weights[count]
  out_m.write(word + "\n")
  out_v.write('\t'.join(str(x) for x in embeddings) + "\n")
out_v.close()
out_m.close()

In [45]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>